In [1]:
import numpy as np
import pandas as pd
import pulp
import json
import requests

### CUSTOM
import sys
sys.path.append('../Modules')
import LpPickTeam

In [2]:
# STANDARDIZED COLUMN LABELS
# Name, Team, Positiom, Cost, [GW]_xP, ID
def standardize_labels(df, old_name:str, old_team:str, old_position:str, old_cost:str, old_xP:str, old_id:str, gw:str):
    return df.rename(columns = {old_name : 'Name', \
                                old_team : 'Team', \
                                old_position : 'Position', \
                                old_cost : 'Cost', \
                                old_xP : gw + '_xP', \
                                old_id : 'ID'})

In [3]:
# GET FPL DATA
url = 'https://fantasy.premierleague.com/api/bootstrap-static/' # API URL
resp = requests.get(url)
data = resp.json()
df = pd.DataFrame(data['elements'])[['web_name', 'team', 'element_type', 'now_cost', 'ep_next', 'id']]
df = df.rename(columns = {'web_name' : 'Name', 'team' : 'Team', 'now_cost' : 'Cost', 'element_type' : 'PosNum', 'ep_next' : '1_xP', 'id' : 
                         'ID'})
# APPLY ONLY ONCE
position_key = {1 : 'GK', 2 : 'DEF', 3: 'MID', 4 : 'FWD'}
df['Position'] = df['PosNum'].apply(lambda num : position_key[num])
df['Cost'] = df['Cost'].apply(lambda price : price / 10)
df['Team'] = df['Team'].apply(lambda team_num : str(team_num))
df['1_xP'] = df['1_xP'].apply(lambda xP : float(xP))
df_official = df[['Name', 'Team', 'Position', 'Cost', '1_xP', 'ID']]

# STANDARDIZED KIWI DATA
df_kiwi = pd.read_csv('../Data/Projected_Points/FPLKiwi_projections.csv')[['Name', 'Team', 'Pos', 'Price', 'xPts 1', 'ID']]
df_kiwi = standardize_labels(df_kiwi, 'Name', 'Team', 'Pos', 'Price', 'xPts 1', 'ID', '1')
df_review = pd.read_csv('../Data/Projected_Points/fplreview_projections.csv')[['Name', 'Team', 'Pos', 'BV', '1_Pts', 'ID']]
df_review = standardize_labels(df_review, 'Name', 'Team', 'Pos', 'BV', '1_Pts', 'ID', '1')

In [4]:
team_key_df = pd.DataFrame(data['teams'])[['id', 'name', 'short_name']]
team_key_df = team_key_df.rename(columns={'id' : 'fpl_id', 'name' : 'long_name', 'short_name' : 'short_name'})
#team_key_df.to_csv('../Dictionaries/team_names.csv', index=False)
position_key = [[1, 'GK', 'G'], [2, 'DEF', 'D'], [3, 'MID', 'M'], [4, 'FWD', 'F']]
position_key_df = pd.DataFrame(position_key, columns=['position_code', 'position_name', 'fplreview_name'])
#position_key_df.to_csv('../Dictionaries/position_names.csv', index=False)

In [5]:
team_dict1 = dict(zip(team_key_df['fpl_id'].apply(str), team_key_df['short_name']))
team_dict2 = dict(zip(team_key_df['long_name'], team_key_df['short_name']))
position_dict2 = dict(zip(position_key_df['fplreview_name'], position_key_df['position_name']))
position_dict1 = dict(zip(position_key_df['position_code'], position_key_df['position_name']))

In [6]:
# STANDARDIZED ENTRY LABEL STYLES
# Name - FPL Web Name
# Team - ShOrt Name
# Position - G, D, M, F
# Cost - In millions
# ID - FPL ID
df_official = df_official.replace({'Team' : team_dict1, 'Position' : position_dict1})
df_review = df_review.replace({'Team' : team_dict2, 'Position' : position_dict2})

In [7]:
# CLEAN BAD
df_review = df_review.query("0 < Cost < 14")
df_kiwi = df_kiwi.query("ID >= 1")

In [8]:
array = []
for ID_n in df_official.ID.unique():
    info1 = df_official[df_official['ID'] == ID_n]
    info2 = df_kiwi[df_kiwi['ID'] == ID_n]
    info3 = df_review[df_review['ID'] == ID_n]
    try:
        row = [info1['Name'].values[0], \
                   info1['ID'].values[0], \
                   info1['Team'].values[0], \
                   info1['Position'].values[0], \
                   info1['Cost'].values[0], \
                   info1['1_xP'].values[0], \
                   info2['1_xP'].values[0], \
                   info3['1_xP'].values[0]]
    except:
        try: 
            row = [info1['Name'].values[0], \
               info1['ID'].values[0], \
               info1['Team'].values[0], \
               info1['Position'].values[0], \
               info1['Cost'].values[0], \
               info1['1_xP'].values[0], \
               info2['1_xP'].values[0], None]
        except:
            try: 
                row = [info1['Name'].values[0], \
                   info1['ID'].values[0], \
                   info1['Team'].values[0], \
                   info1['Position'].values[0], \
                   info1['Cost'].values[0], \
                   info1['1_xP'].values[0], \
                   info2['1_xP'].values[0], info3['1_xP'].values[0]]
            except:
                row = [info1['Name'].values[0], \
                   info1['ID'].values[0], \
                   info1['Team'].values[0], \
                   info1['Position'].values[0], \
                   info1['Cost'].values[0], \
                   info1['1_xP'].values[0], \
                   None, None]
    array.append(row)
all_data = pd.DataFrame(array, columns=['Name', 'ID', 'Team', 'Position', 'Cost', 'OfficialFPLSite', 'theFPLKiwi', 'FPLReview'])
all_data['Average'] = 1/3 * (sum(all_data[place] for place in ['OfficialFPLSite', 'theFPLKiwi', 'FPLReview']))

In [9]:
cleaned_df = all_data.sort_values(by='Average', ascending=False)

In [10]:
cleaned_df.to_csv('../Data/combined_projections.csv', index=False)

In [11]:
cleaned_df.sort_values(by='Average', ascending=False).head(n=50)

,Name,ID,Team,Position,Cost,OfficialFPLSite,theFPLKiwi,FPLReview,Average
298,Salah,283,LIV,MID,13.0,5.5,6.28,7.38,6.386667
454,Kane,427,TOT,FWD,11.5,4.5,6.44,6.46,5.800000
455,Son,428,TOT,MID,12.0,4.2,6.38,6.37,5.650000
300,Alexander-Arnold,285,LIV,DEF,7.5,5.5,5.47,5.64,5.536667
320,De Bruyne,301,MCI,MID,12.0,4.7,4.85,5.00,4.850000
335,Haaland,318,MCI,FWD,11.5,5.0,4.13,5.42,4.850000
299,Robertson,284,LIV,DEF,7.0,5.2,4.41,4.90,4.836667
296,Alisson,281,LIV,GK,5.5,5.5,4.16,4.27,4.643333
350,Fernandes,333,MUN,MID,10.0,3.7,5.10,4.91,4.570000
324,Cancelo,306,MCI,DEF,7.0,4.7,4.84,3.87,4.470000


In [12]:
help(LpPickTeam.pick_team)

Help on function pick_team in module LpPickTeam:

pick_team(xPdf, criterion: str, formation: tuple[int], total_cost: float)
    Finds optimal FPL team from projected expected points.
    
    xPdf -- data frame with cost, xP, team, position, ID, and name information
    criterion -- columns which optimizer uses for xP players
    formation -- football formation used for starting 11
    total_cost -- maximum cost of squad



In [13]:
cleaned_df.sort_values(by='Average', ascending=False).query("Average > 0 ")

,Name,ID,Team,Position,Cost,OfficialFPLSite,theFPLKiwi,FPLReview,Average
298,Salah,283,LIV,MID,13.0,5.5,6.28,7.38,6.386667
454,Kane,427,TOT,FWD,11.5,4.5,6.44,6.46,5.800000
455,Son,428,TOT,MID,12.0,4.2,6.38,6.37,5.650000
300,Alexander-Arnold,285,LIV,DEF,7.5,5.5,5.47,5.64,5.536667
320,De Bruyne,301,MCI,MID,12.0,4.7,4.85,5.00,4.850000
...,...,...,...,...,...,...,...,...,...
420,Richards,511,NFO,DEF,4.5,0.0,0.08,0.01,0.030000
207,Branthwaite,196,EVE,DEF,4.0,0.0,0.06,0.00,0.020000
101,Bidstrup,99,BRE,MID,4.5,0.0,0.04,0.00,0.013333
127,Richards,121,BHA,MID,4.5,0.0,0.04,0.00,0.013333


In [41]:
solution = LpPickTeam.pick_team(cleaned_df.sort_values(by='Average', ascending=False).query("Average > 0 "), 'Average', (4, 4, 2), 96)

In [42]:
print('Squad : \n', solution[1].to_string(), '\n', 'xScore : ', solution[2])

Squad : 
                          Name   ID Team Position  Cost        xP
Starters 0               Kane  427  TOT      FWD  11.5  5.800000
         1   Alexander-Arnold  285  LIV      DEF   7.5  5.536667
         2          Robertson  284  LIV      DEF   7.0  4.836667
         3            Alisson  281  LIV       GK   5.5  4.643333
         4            Cancelo  306  MCI      DEF   7.0  4.470000
         5              James  146  CHE      DEF   6.0  4.116667
         6             Wilson  356  NEW      FWD   7.5  4.063333
         7              Mount  142  CHE      MID   8.0  4.053333
         8      Saint-Maximin  368  NEW      MID   6.5  3.653333
         9    Bruno Guimarães  374  NEW      MID   6.0  3.463333
         10          Rashford  335  MUN      MID   6.5  3.210000
Bench    0           Balcombe  100  BRE       GK   4.0  0.343333
         1             Plange  496  CRY      FWD   4.5  0.253333
         2           Bidstrup   99  BRE      MID   4.5  0.013333
         3     

In [25]:
pd.DataFrame(solution[0]['Starters'])

,var,Cost,xP,Team,Position,Name,ID
0,Starters_298,13.0,6.386667,LIV,MID,Salah,283
1,Starters_454,11.5,5.800000,TOT,FWD,Kane,427
2,Starters_455,12.0,5.650000,TOT,MID,Son,428
3,Starters_300,7.5,5.536667,LIV,DEF,Alexander-Arnold,285
4,Starters_320,12.0,4.850000,MCI,MID,De Bruyne,301
...,...,...,...,...,...,...,...
495,Starters_420,4.5,0.030000,NFO,DEF,Richards,511
496,Starters_207,4.0,0.020000,EVE,DEF,Branthwaite,196
497,Starters_101,4.5,0.013333,BRE,MID,Bidstrup,99
498,Starters_127,4.5,0.013333,BHA,MID,Richards,121


In [28]:
solution[1].stack()

Starters  0  Name            Kane
             ID               427
             Team             TOT
             Position         FWD
             Cost            11.5
                           ...   
Bench     3  ID               321
             Team             SOU
             Position         MID
             Cost             4.5
             xP          0.223333
Length: 90, dtype: object

# Dependencies
- numpy
- pandas
- pulp
- json
- requests

# Data Sources
## Projected Points
-  Hi 